In [147]:
import pandas as pd

df_raw = pd.read_csv('recipes.csv')

print(df_raw.columns)
print(df_raw.head(2).to_dict())

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')
{'RecipeId': {0: 38, 1: 39}, 'Name': {0: 'Low-Fat Berry Blue Frozen Dessert', 1: 'Biryani'}, 'AuthorId': {0: 1533, 1: 1567}, 'AuthorName': {0: 'Dancer', 1: 'elly9812'}, 'CookTime': {0: 'PT24H', 1: 'PT25M'}, 'PrepTime': {0: 'PT45M', 1: 'PT4H'}, 'TotalTime': {0: 'PT24H45M', 1: 'PT4H25M'}, 'DatePublished': {0: '1999-08-09T21:46:00Z', 1: '1999-08-29T13:12:00Z'}, 'Description': {0: 'Make and share this Low-Fat Berry Blue Frozen Dessert recipe from Food.com.', 1: 'Make

In [148]:
# Pick the columns to keep and rename them
columns_to_keep = {
    'Name': 'title',
    'RecipeCategory': 'category',
    'Keywords': 'keywords',
    'RecipeServings': 'servings',
    'RecipeIngredientParts': 'ingredients',
    'RecipeIngredientQuantities': 'ingredient_quantities',
    'RecipeInstructions': 'steps',
    'AggregatedRating': 'rating',
    'ReviewCount': 'review_count',
#    'Description': 'description',
#    'CookTime': 'cook_time',
#    'PrepTime': 'prep_time',
#    'TotalTime': 'total_time',
#    'RecipeId': 'recipe_id',
#    'AuthorId': 'author_id',
#    'AuthorName': 'author_name',
#    'DatePublished': 'date_published',
#    'Images': 'images',
#    'Calories': 'calories',
#    'FatContent': 'fat_content',
#    'SaturatedFatContent': 'saturated_fat_content',
#    'CholesterolContent': 'cholesterol_content',
#    'SodiumContent': 'sodium_content',
#    'CarbohydrateContent': 'carbohydrate_content',
#    'FiberContent': 'fiber_content',
#    'SugarContent': 'sugar_content',
#    'ProteinContent': 'protein_content',
#    'RecipeYield': 'recipe_yield'
}

# Select and rename the columns
df = df_raw[list(columns_to_keep.keys())].rename(columns=columns_to_keep)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   title                  522517 non-null  object 
 1   category               521766 non-null  object 
 2   keywords               505280 non-null  object 
 3   servings               339606 non-null  float64
 4   ingredients            522517 non-null  object 
 5   ingredient_quantities  522514 non-null  object 
 6   steps                  522517 non-null  object 
 7   rating                 269294 non-null  float64
 8   review_count           275028 non-null  float64
dtypes: float64(3), object(6)
memory usage: 35.9+ MB


In [149]:
def calculate_missing_percentage(df):
    # Calculate the percentage of missing values for each column and print them
    missing_percentage = df.isna().mean() * 100
    missing_info = missing_percentage.sort_values(ascending=False).reset_index()
    missing_info.columns = ['Column', 'Percentage of Missing Values']
    print(missing_info)

calculate_missing_percentage(df)


                  Column  Percentage of Missing Values
0                 rating                     48.462155
1           review_count                     47.364775
2               servings                     35.005751
3               keywords                      3.298840
4               category                      0.143727
5  ingredient_quantities                      0.000574
6                  title                      0.000000
7            ingredients                      0.000000
8                  steps                      0.000000


In [150]:
# Normalize text columns
df['title'] = df['title'].str.strip().str.lower()
df['category'] = df['category'].str.strip().str.lower()
df['keywords'] = df['keywords'].str.strip().str.lower()
df['ingredients'] = df['ingredients'].str.strip().str.lower()
df['ingredient_quantities'] = df['ingredient_quantities'].str.strip().str.lower()
df['steps'] = df['steps'].str.strip().str.lower()

# Ensure 'servings', 'rating', 'review_count' are numeric
df['servings'] = pd.to_numeric(df['servings'], errors='coerce')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['review_count'] = pd.to_numeric(df['review_count'], errors='coerce')

df = df.dropna(subset=['title', 'category', 'keywords', 'servings', 'ingredients', 'ingredient_quantities', 'steps'])
df.info()
calculate_missing_percentage(df)

<class 'pandas.core.frame.DataFrame'>
Index: 327969 entries, 0 to 522514
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   title                  327969 non-null  object 
 1   category               327969 non-null  object 
 2   keywords               327969 non-null  object 
 3   servings               327969 non-null  float64
 4   ingredients            327969 non-null  object 
 5   ingredient_quantities  327969 non-null  object 
 6   steps                  327969 non-null  object 
 7   rating                 168216 non-null  float64
 8   review_count           171601 non-null  float64
dtypes: float64(3), object(6)
memory usage: 25.0+ MB
                  Column  Percentage of Missing Values
0                 rating                     48.709787
1           review_count                     47.677677
2                  title                      0.000000
3               category               

In [151]:
import ast

# Format the list to a python list - return None if list is empty
def convert_to_list(item_str):
    try: # Remove 'c(' and ')' and convert the string into a list
        cleaned_str = item_str.lstrip('c(').rstrip(')').replace('NA', '"to taste"')
        return ast.literal_eval(f'[{cleaned_str}]')
    
    except (ValueError, SyntaxError): # Return none if there are issues, to drop later
        return None

# Apply this function directly to the relevant columns
df['keywords'] = df['keywords'].apply(convert_to_list)
df['ingredients'] = df['ingredients'].apply(convert_to_list)
df['ingredient_quantities'] = df['ingredient_quantities'].apply(convert_to_list)
df['steps'] = df['steps'].apply(convert_to_list)

In [152]:
df = df.dropna(subset=['keywords','ingredients', 'ingredient_quantities', 'steps'])
calculate_missing_percentage(df)

                  Column  Percentage of Missing Values
0                 rating                     49.044200
1           review_count                     47.977734
2                  title                      0.000000
3               category                      0.000000
4               keywords                      0.000000
5               servings                      0.000000
6            ingredients                      0.000000
7  ingredient_quantities                      0.000000
8                  steps                      0.000000


,title,category,keywords,servings,ingredients,ingredient_quantities,steps,rating,review_count
0,low-fat berry blue frozen dessert,frozen desserts,"[dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]",4.0,"[blueberries, granulated sugar, vanilla yogurt, lemon juice]","[4, 1/4, 1, 1]","[toss 2 cups berries with sugar., let stand for 45 minutes, stirring occasionally., transfer berry-sugar mixture to food processor., add yogurt and process until smooth., strain through fine sieve. pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). freeze uncovered until edges are solid but centre is soft. transfer to processor and blend until smooth again., return to pan and freeze until edges are solid., transfer to processor and blend until smooth again., fold in remaining 2 cups of blueberries., pour into plastic mold and freeze overnight. let soften slightly to serve.]",4.5,4.0
3,carina's tofu-vegetable kebabs,soy/tofu,"[beans, vegetable, low cholesterol, weeknight, broil/grill, oven]",2.0,"[extra firm tofu, eggplant, zucchini, mushrooms, soy sauce, low sodium soy sauce, olive oil, maple syrup, honey, red wine vinegar, lemon juice, garlic cloves, mustard powder, black pepper]","[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4]","[drain the tofu, carefully squeezing out excess water, and pat dry with paper towels., cut tofu into one-inch squares., set aside. cut eggplant lengthwise in half, then cut each half into approximately three strips., cut strips crosswise into one-inch cubes., slice zucchini into half-inch thick slices., cut red pepper in half, removing stem and seeds, and cut each half into one-inch squares., wipe mushrooms clean with a moist paper towel and remove stems., thread tofu and vegetables on to barbecue skewers in alternating color combinations: for example, first a piece of eggplant, then a slice of tofu, then zucchini, then red pepper, baby corn and mushrooms., continue in this way until all skewers are full., make the marinade by putting all ingredients in a blender, and blend on high speed for about one minute until mixed., alternatively, put all ingredients in a glass jar, cover tightly with the lid and shake well until mixed., lay the kebabs in a long, shallow baking pan or on a non-metal tray, making sure they lie flat. evenly pour the marinade over the kebabs, turning them once so that the tofu and vegetables are coated., refrigerate the kebabs for three to eight hours, occasionally spooning the marinade over them., broil or grill the kebabs at 450 f for 15-20 minutes, or on the grill, until the vegetables are browned., suggestions this meal can be served over cooked, brown rice. amounts can easily be doubled to make four servings.]",4.5,2.0
4,cabbage soup,vegetable,"[low protein, vegan, low cholesterol, healthy, winter, < 60 mins, easy]",4.0,"[plain tomato juice, cabbage, onion, carrots, celery]","[46, 4, 1, 2, 1]","[mix everything together and bring to a boil., reduce heat and simmer for 30 minutes (longer if you prefer your veggies to be soft)., refrigerate until cool., serve chilled with sour cream.]",4.5,11.0
7,buttermilk pie with gingersnap crumb crust,pie,"[dessert, healthy, weeknight, oven, < 4 hours]",8.0,"[sugar, margarine, egg, flour, salt, buttermilk, graham cracker crumbs, margarine]","[3/4, 1, 1, 2, 3, 1/4, 1, 1/2, 1/2, 2]","[preheat oven to 350°f., make pie crust, using 8 inch pie pan, do not bake., mix sugar and margarine in medium bowl until blended; beat in egg whites and egg., stir in flour, salt, and buttermilk until well blended., pour filling into prepared crust, bake 40 minutes or until sharp knife inserted near center comes out clean., sprinkle with nutmeg and serve warm or chilled., combine graham crumbs, gingersnap crumbs, and margarine in 8 or 9 inch pie pan, pat mixture evenly on bottom and side of pan., bake 8 to 10 minutes or until edge of crust is lightly browned., cool on wire rack.]",4

In [153]:
# Flatten the ingredients lists
all_ingredients = []
for ingredients_list in df['ingredients']:
    # Directly flatten and append the ingredients
    all_ingredients.extend(ingredients_list)

# all_ingredients_normalized = [ingredient.lower().strip() for ingredient in all_ingredients]

# Get unique ingredients
unique_ingredients = sorted(set(all_ingredients))
# unique_ingredients_Notnormal = sorted(set(all_ingredients))


# print (len(unique_ingredients_Notnormal))
print (len(unique_ingredients))
# Output the unique ingredients
for ingredient in unique_ingredients:
    print(ingredient)

# Convert the list to a pandas DataFrame and save it to a CSV file

unique_ingredients_df = pd.DataFrame(unique_ingredients, columns=['Ingredients'])
#unique_ingredients_df.to_csv('unique_ingredients_list.csv', index=False)



5819
1% fat buttermilk
1% fat cottage cheese
1% low-fat chocolate milk
1% low-fat milk
1-1/2 ingredient fiber crust
10-inch corn tortillas
10-inch flour tortilla
10-inch flour tortillas
10-inch whole wheat  tortillas
10-minute success rice
100 proof vodka
12-inch flour tortilla
12-inch flour tortillas
2 texans craving salsa far from home
2% buttermilk
2% cheddar cheese
2% evaporated milk
2% fat cottage cheese
2% low-fat chocolate milk
2% low-fat milk
2% milk
2% mozzarella cheese
20% sour cream
2bleu's 2 minute 2 easy pizza sauce
2bleu's sweet mustard sauce for pretzels and more!
3 legume butter
3-cheese gourmet cheddar blend cheese
5% fat ricotta cheese
6-inch corn tortillas
6-inch flour tortillas
6-inch tortillas
7-inch corn tortillas
7-inch flour tortillas
8-inch 97% fat free flour tortillas
8-inch fat-free flour tortillas
8-inch flour tortillas
8-inch low-carb whole wheat tortilla
8-inch ready-made graham cracker crust
9-inch flour tortillas
9-inch graham cracker crust
9-inch graham

In [154]:
unique_categories = df['category'].unique()

print(len(unique_categories))

drop_keywords = [
    'pie', 'cake', 'dessert', 'cheesecake', 'candy', 'cookie', 'gelatin', 
    'baking', 'sweets', 'pastries', 'pudding', 'cupcakes', 'brownies', 'mousse', 
    'tarts', 'muffins', 'turnovers', 'ice cream', 'bread', 'beverage', 'smoothie', 
    'shakes', 'alcohol', 'cocktail', 'tea', 'coffee', 'wine', 'beer', 'dressing'
]

# Drop deserts, drinks and baking recipes ( leaving mostly meals )
df = df[~df['category'].str.contains('|'.join(drop_keywords), case=False, na=False)]

# drop all without rating 
df = df.dropna(subset=['rating', 'review_count'])

# drop unless rating > 4 and more than 5 reviews 
df = df[df['rating'] >= 4.5]
df = df[df['review_count'] >= 5]

# drop where servings are not between 1-10
df = df[(df['servings'] >= 1) & (df['servings'] <= 10)]

# drop all with less than 3 keywords 
df = df[df['keywords'].apply(lambda x: isinstance(x, list) and len(x) >= 3)]

# Merge the keywords and category to a "tags" list 
df['keywords'] = df.apply(lambda row: row['keywords'] + [row['category']], axis=1)
df = df.rename(columns={'keywords': 'tags'})
df = df.drop(columns=['category'])

df

282


,title,tags,servings,ingredients,ingredient_quantities,steps,rating,review_count
4,cabbage soup,"[low protein, vegan, low cholesterol, healthy, winter, < 60 mins, easy, vegetable]",4.0,"[plain tomato juice, cabbage, onion, carrots, celery]","[46, 4, 1, 2, 1]","[mix everything together and bring to a boil., reduce heat and simmer for 30 minutes (longer if you prefer your veggies to be soft)., refrigerate until cool., serve chilled with sour cream.]",4.5,11.0
11,chicken breasts lombardi,"[chicken, poultry, meat, european, very low carbs, weeknight, oven, < 4 hours, chicken breast]",6.0,"[fresh mushrooms, butter, boneless skinless chicken breast halves, flour, butter, marsala, chicken broth, salt, mozzarella cheese, parmesan cheese, green onion]","[2, 2, 12, 1/2, 1/3, 3/4, 1/2, 1/2, 1/2, 1/2, 1/4]","[cook mushrooms in 2 tbsp butter in a large skillet, stirring constantly, just until tender., remove from heat; set aside. cut each chicken breast half in half lengthwise. place each piece of chicken between two sheets of wax paper; flatten to 1/8"" thickness, using a meat mallet or rolling pin., dredge chicken pieces in flour., place 5 or 6 pieces of chicken in 1 to 2 tbsp butter in a large skillet; cook over medium heat 3 to 4 minutes on each side or until golden., place chicken in a lightly greased 13x9"" baking dish, overlapping edges., repeat procedure with remaining chicken and butter., reserve pan drippings in skillet. sprinkle reserved mushrooms over chicken., add wine and broth to skillet. bring to a boil; reduce heat, and simmer, uncovered, 10 minutes, stirring occasionally., stir in salt and pepper., pour sauce over chicken., combine cheeses and green onions; sprinkle over chicken., bake uncovered at 450 for 12 to 14 minutes., broil 5 1/2"" away from heat 1 to 2 minutes or until browned.]",5.0,21.0
20,low-fat burgundy beef & vegetable stew,"[vegetable, meat, low cholesterol, healthy, free of..., weeknight, < 4 hours, stew]",6.0,"[beef eye round, dried thyme leaves, salt, pepper, ready-to-serve beef broth, burgundy wine, garlic, cornstarch, frozen sugar snap peas]","[1 1/2, 1, 1, 1/2, 1/2, 1, 1/2, 3, 5 1/2, 1, 2, 1]","[trim fat from beef, cut into 1-inch pieces., in dutch oven, heat oil over medium high hunt until hot. add beef (half at a time) and brown evenly, stirring occasionally., pour off drippings., season with thyme, salt and pepper., stir in broth, wine and garlic. bring to boil; reduce heat to low., cover tightly and simmer 1 1/2 hours., add carrots and onions., cover and continue cooking 35 to 40 minutes or until beef and vegetables are tender., bring beef stew to a boil over medium-high heat. add cornstarch mixture; cook and stir 1 minute. stir in sugar snap peas., reduce heat to medium and cook 3 to 4 minutes or until peas are heated through.]",4.5,7.0
24,"black bean, corn, and tomato salad","[corn, beans, vegetable, low cholesterol, healthy, free of..., spring, summer, < 30 mins, easy, black beans]",2.0,"[fresh lemon juice, olive oil, black beans, fresh corn kernels, plum tomato, scallion, fresh parsley leaves, cayenne, boston lettuce leaves]","[3, 2, 1, 1, 1, 1, 2, 1, 4]","[in a bowl whisk together lemon juice, oil, and salt to taste., stir in remaining ingredients, except lettuce leaves, with salt and black pepper to taste., let salad stand, stirring once or twice, 15 minutes for flavors to develop., line 2 plates with lettuce and divide salad between them.]",5.0,23.0
51,brown rice and vegetable pilaf,"[rice, vegetable, weeknight, < 4 hours, brown rice]",6.0,"[brown rice, chicken broth, unsalted butter, oregano, marjoram, summer savory, unsalted butter, celery, fresh snow pea, broccoli floret, green onion, lemons, coconut]","[1 1/2, 3, 3, 1/2, 1/2, 1/2, 5, 1, 1, 1, 1, 1 1/2, 2, 3/4]","[preheat oven to 325 degrees f., heat chicken broth to boiling., combine broth with brown rice, butter, oregano, marjoram, and summery savory in 3 quart casserole and bake, covered, for 1-1/2 hours or until rice is tender., keep

In [161]:
# Finally trim it to a 5k rows (for more manageable training)
# And eport 
trimmed_df = df.sample(n=5000, random_state=50)
trimmed_df.to_csv('recipes_5k_pruned.csv', index=False)
trimmed_df

,title,tags,servings,ingredients,ingredient_quantities,steps,rating,review_count
219751,spicy hot dog bites,"[summer, beginner cook, < 4 hours, easy, meat]",4.0,"[ketchup, brown sugar, red wine, white vinegar, soy sauce, dijon mustard, spicy brown mustard, cayenne, garlic clove]","[1, 1/2, 1, 2, 1, 1/2, 1, 1]","[combine ketchup, brown sugar, vinegar, soy sauce, mustard, and garlic in the crockery pot., cover and cook on high for 1 hour, stirring occasionally., add hot dogs; stir to coat with sauce., cover and cook on low for 1 1/2 hours, or until hot., serve hot, with toothpicks, from crockpot.]",4.5,6.0
267038,ham &amp; pineapple pizza muffins,"[pineapple, pork, tropical fruits, fruit, meat, kid friendly, < 30 mins, easy, ham]",8.0,"[english muffins, tomato paste, dried oregano, pineapple chunks in juice, ham, mozzarella cheese]","[4, 1/2, 1, 440, 150, 200]","[split each muffin in half, spread each with the paste, top each muffin with the pineapple, ham then cheese and oregano. place muffins on oven tray in moderate oven for about 15 - 20 minutes, or until cheese is melted and pizzas are heated through., serve immediately.]",5.0,5.0
195653,cheesy chicken and rice casserole,"[chicken breast, chicken, long grain rice, poultry, rice, cheese, vegetable, meat, kid friendly, spring, < 60 mins, oven, easy, inexpensive, one dish meal]",4.0,"[water, long-grain white rice, onion powder, boneless skinless chicken breast halves, cheddar cheese]","[1, 1 1/3, 3/4, 2, 1/2, 4, 1/2]","[stir the soup, water, rice, vegetables and onion powder in a 12"" x 8"" shallow baking dish., top with chicken., season chicken as desired., cover., bake at 375°f for 45 minutes or until done., top with cheese. makes 4 servings.]",4.5,10.0
13382,eggplant (aubergine) kuku (persian eggplant),"[vegetable, southwest asia (middle east), asian, low protein, kosher, potluck, < 4 hours, lunch/snacks]",4.0,"[eggplants, eggplants, ghee, onions, garlic cloves, eggs, fresh parsley, powdered saffron, lime, juice of, baking powder, all-purpose flour, salt, fresh ground black pepper]","[2, 1, 1/2, 2, 4, 4, 4, 1/4, 1, 1, 1, 1, 1/4]","[peel the eggplants, cut them lengthwise in quarters if they are large, and salt them to remove bitternes if necessary., brush each side of the eggplant pieces with eggwhite to reduce the oil needed for frying., in a skillet, heat 4 tablespoons oil over medium heat., add the onion and stir-fry for 10 minutes, until translucent., add the eggplant and garlic and stir-fry 10 minutes longer, until all sides are lightly golden brown., remove from heat and allow to cool., preheat the oven to 350°f., pour 4 tablespoons of oil into an 8-inch baking dish lined with parchment paper., break the eggs into a large bowl., add the parsley, saffron water, lime juice, baking powder, flour, salt, and pepper., beat thoroughly with a fork., add the eggplant, onion and garlic and mix thoroughly., pour the mixture into the dish and bake uncovered for 45 to 50 minutes, until the edge is golden brown., serve the kuku from the baking dish or unmold it by loosening the edge with a knife and inverting the dish onto a serving platter., remove the parchment paper.]",4.5,13.0
254685,homemade sloppy joes or hot dog chili,"[meat, kid friendly, < 30 mins, beginner cook, easy, inexpensive, lunch/snacks]",4.0,"[lean ground beef, ketchup, water, chili powder, paprika, salt, onion powder, black pepper]","[1, 1/2, 1 1/2, 1/2, 1 - 1 1/2, 1/2, 1/2, 1/2, 1/4, 8]","[brown the beef in a large frying pan, breaking it into tiny pieces., drain any excess fat then stir in the rest of the ingredients (except for the buns) with the beef. simmer on medium heat until the mixture is thick, about 10 minutes., serve on hamburger buns or over hot dogs.]",5.0,9.0
...,...,...,...,...,...,...,...,...
62264,pear and prosciutto di parma salad,"[pork, cheese, pears, greens, fruit, vegetable, nuts, meat, european, savory, sweet, < 30 mins, stove top, ham]",4.0,"[butter, boston lettuce, pears, prosciutto di p